# 스파크 기능 둘러보기
- 스파크의 기본 요소
    - 저수준 API
    - 구조적 API
    - 추가 기능을 제공하는 일련의 표준 라이브러리
![spark-function-structure](../../assets/presentations/week03/sparks_toolset.png)

- 스파크 라이브러리는 그래프 분석, 머신러닝 그리고 스트리밍 등 다양한 작업을 지원하고, 컴퓨팅 및 스토리지 시스템과의 통합을 돕는 역할을 한다.

## 1. spark-submit
- `spark-submit`을 통해 사용자 애플리케이션을 Spark 클러스터에 전송하고 실행시킬 수 있다.
- aws 참고: https://aws.amazon.com/ko/blogs/korea/submitting-user-applications-with-spark-submit/
- 클러스터에서 애플리케이션을 실행하는 일반적인 방법으로 spark-submit 스크립트를 사용하는 것이다.
- spark-submit 스크립트는 애플리케이션에서 사용하는 리소스를 제어할 수 있는 여러 플래그를 제공한다.

```shell
$ spark-submit --help
Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
...
```


### 예시
1. 스파크가 지원하는 프로그래밍 언어로 애플리케이션을 개발한다.

```shell
# 스칼라로 작성한 애플리케이션 실행
./bin/spark-submit \
--class org.apache.spark.examples.SparkPi \
--master local \
./examples/jars/spark-examples_2.11-2.2.0.jar 10

# 파이썬으로 작성한 애플리케이션 실행
./bin/spark-submit \
--master local \
./examples/src/main/python/pi.py 10
```

2. 위의 예제는 `--master local` 로컬 머신에서 애플리케이션을 실행한다.
    - **master 옵션의 인숫값을 변경**해 스파크가 지원하는 스파크 스탠드얼론, 메소스, YARN 클러스터 매니저에서 애플리케이션을 실행할 수 있다. 

## 2. 구조적 API - Dataset
#### 타입 안정성을 제공하는 구조적 API Dataset

*Dataset은 타입 안정성을 지원하며 동적 타입 언어인 파이썬과 R에서는 사용이 불가하다.*

#### RDD, DataFrame, Dataset..에 대해서 
- 참고 wikidocs: https://wikidocs.net/28377

![History of Spark APIs](../../assets/presentations/week03/history_spark.png)

- RDD
    - 2014년 정식 출시된 스파크 v1은 RDD API를 이용하여 데이터를 처리. 테이블 조인 효율화 같은 처리를 사용자가 직접 제어해야 하는 단점이 존재
- DataFrame
    - 스파크 1.3d에서 처리 속도 증가를 위한 프로젝트 텅스텐의 일부로 소개 되었으며, 데이터를 스키마 형태로 추상화하고, 카탈리스트 옵티마이저가 쿼리를 최적화하여 처리한다.
    - 다양한 데이터 타입의 테이블형 데이터를 보관할 수 있는 Row 타입의 객체로 구성된 분산 컬렉션이다.
- Dataset
    - 스파크 1.6에서 추가되었으며, 데이터의 타입체크, 데이터 직렬화를 위한 인코더, 카탈리스트 옵티마이저를 지원하여 데이터 처리 속도를 더욱 증가시켰다.
    - 스칼라에서는 데이터프레임과 데이터셋을 통합시켰으며, 스칼라 API에서는 Dataset[Row]는 DataFrame을 의미한다.
    - Dataset API는 DataFrame의 레코드를 사용자가 자바나 스칼라로 정의한 클래스에 할당하고 자바의 ArrayList 또는 스칼라의 Seq 객체 등의 고정 타입형 컬렉션으로 다룰 수 있는 기능을 제공한다.
    - Dataset API는 **타입 안정성을 지원**하므로 초기화에 사용한 클래스 대신 다른 클래스를 사용해 접근할 수 없다.
    
    
- 스파카 애플리케이션 개발 시, RDD는 스파크 컨텍스트(SparkContext)를 이용하고, 데이터셋과 데이터프레임은 스파크 세션(SparkSession) 객체를 이용한다.


### Dataset 장점
- 필요한 경우에 선택적으로 사용할 수 있다.
- collect 메서드나 take 메서드를 호출하면 DataFrame을 구성하는 Row 타입의 객체가 아닌 ㅇDataset에 매개변수로 지정한 타입의 객체를 반환한다. 코드 변경 없이 타입 안정성을 보장할 수 있으며, 어떤한 환경에서든 데이터를 안전하게 다룰 수 있다.

## 3. 구조적 스트리밍: 스트림 처리용 고수준 API
- 구조적 스트리밍을 사용하면 구조적 API로 개발된 **배치 모드의 연산을 스트리밍 방식으로 실행**할 수 있다.
- 지연 시간을 줄이고 증분 처리할 수 있다. 모든 작업은 데이터를 증분 처리하면서 수행된다.
- 배치 처리용 코드를 일부 수정하여 스트리밍 처리를 수행하고 값을 빠르게 얻을 수 있다는 장점이 있다. 


### 배치 처리 VS 스트리밍 처리
- `read` 메서드 대신 `readStream` 메서드 사용
- 스트리밍 액션은 일반적인 정적 액션과는 다르다.
    - 스트리망 액션은 **트리거**가 실행된 다음 데이터를 갱신하게 될 인메모리 테이블에 데이터를 저장한다. 
    - 이전 집계값보다 더 큰 값이 발생한 경우에만 인메모리 테이블을 갱신하므로 언제나 가장 큰 값을 얻을 수 있다.
    - 스파크가 데이터를 처리하는 시점이 아닌 이벤트 시간에 따라 윈도우를 구성하는 방식에 주목할 필요가 있다(?)


#### exercises
- ../../exercises/week03/batch-example.ipynb
- ../../exercises/week03/streaming-example.ipynb

## 4. 머신러닝과 고급 분석
- 스파크에 내장된 머신러닝 알고리즘 라이브러리 `MLlib` 대규모 머신러닝 수행이 가능하다.
- MLlib를 사용해서 preprocessing, munging, model traning, prediction을 할 수 있다.
- 구조적 스트리밍에서 예측하고자 할 때도 MLlib에서 학습시킨 다양한 예측 모델을 사용할 수 있다.


#### exercises
- ../../exercises/week03/MLlib-example.ipynb


## 5. 저수준 API (Lower-Level APIs)
- 스파크는 RDD를 통해 자바와 파이썬 객체를 다루는 데 필요한 다양한 기본 기능을 제공한다.
- **스파크의 거의 모든 기능은 RDD 기반으로 만들어졌다.**
- DataFrame 연산도 RDD 기반으로 만들어졌으며, 효율적인 분산 처리를 위해 저수준 명령으로 컴파일된다.(*원시 데이터를 읽거나 다루는 용도로 RDD를 사용할 수 있지만 대부분은 구조적 API를 사용하는 것이 바람직하다.*)

- RDD를 이용해 파티션과 같은 물리적 실행 특성을 결정할 수 있으므로 DataFrame보다 더 세밀한 제어를 할 수 있다.


## 6. SparkR
- sparkR은 스파크를 R 언어로 사용하기 위한 기능
